# Librerias y Lectura Archivos

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input

In [2]:
#importacion df_final
df=pd.read_csv('../data/processed/final_merge.csv')

In [3]:
df

,edad,sexo,nivel_educativo,fumador,actividad_fisica,vive,diabetes,hipertension,obesidad,cancer,...,region_Sur,tipo_empleo_Desempleado,tipo_empleo_Privado,tipo_empleo_Público,tipo_empleo_Temporal,continente_America,continente_Antarctica,continente_Asia,continente_Europe,continente_Oceania
0,69,0,1,1,0,1,1,1,1,0,...,0,0,0,1,0,0,0,1,0,0
1,32,1,2,0,1,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0
2,89,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,78,1,1,1,0,0,1,0,1,1,...,0,0,0,0,1,0,0,1,0,0
4,38,0,3,1,1,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47605,21,1,2,0,2,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
47606,35,0,3,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
47607,46,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
47608,56,1,3,1,1,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [4]:
df.shape

(47610, 48)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47610 entries, 0 to 47609
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   edad                     47610 non-null  int64  
 1   sexo                     47610 non-null  int64  
 2   nivel_educativo          47610 non-null  int64  
 3   fumador                  47610 non-null  int64  
 4   actividad_fisica         47610 non-null  int64  
 5   vive                     47610 non-null  int64  
 6   diabetes                 47610 non-null  int64  
 7   hipertension             47610 non-null  int64  
 8   obesidad                 47610 non-null  int64  
 9   cancer                   47610 non-null  int64  
 10  enfermedad_cardiaca      47610 non-null  int64  
 11  epoc                     47610 non-null  int64  
 12  glucosa                  47610 non-null  float64
 13  colesterol               47610 non-null  float64
 14  trigliceridos         

# Estandarización Variables

In [6]:
numerical_columns = ['edad', 'glucosa', 'colesterol', 'trigliceridos', 
                     'hemoglobina', 'leucocitos', 'plaquetas', 'creatinina', 
                     'ingresos_mensuales', 'gastos_salud']

In [7]:
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
df[numerical_columns].head()

,edad,glucosa,colesterol,trigliceridos,hemoglobina,leucocitos,plaquetas,creatinina,ingresos_mensuales,gastos_salud
0,0.752464,1.182348,1.148022,0.538101,-0.269240,-0.948031,0.209374,-0.325816,1.525607,2.521666
1,-1.028350,-1.578028,-1.785944,-1.386118,0.182589,-0.084302,0.732393,-0.904565,2.597109,-0.876769
2,1.715066,-0.430133,-0.698475,-2.106839,-0.762145,0.494905,-0.120001,2.664386,1.103434,-0.876769
3,1.185635,-0.245735,0.305460,-0.964549,-0.567037,1.531380,-0.126206,1.072827,-0.003326,2.521666
4,-0.739569,-1.981947,-1.579386,-2.626722,-1.193437,1.165565,-0.402064,-0.759878,0.018125,-0.876769


## Red Neuronal

In [8]:
X = df.drop(columns='vive')
y = df['vive']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelo Simple

In [10]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # salida binaria
])


In [11]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

In [12]:
# 6. Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
953/953 ━━━━━━━━━━━━━━━━━━━━ 1s 616us/step - accuracy: 0.8621 - loss: 0.3780 - precision: 0.8743 - recall: 0.9836 - val_accuracy: 0.8733 - val_loss: 0.3301 - val_precision: 0.8780 - val_recall: 0.9926
Epoch 2/10
953/953 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.8708 - loss: 0.3354 - precision: 0.8741 - recall: 0.9948 - val_accuracy: 0.8733 - val_loss: 0.3257 - val_precision: 0.8773 - val_recall: 0.9937
Epoch 3/10
953/953 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.8723 - loss: 0.3283 - precision: 0.8756 - recall: 0.9944 - val_accuracy: 0.8719 - val_loss: 0.3239 - val_precision: 0.8757 - val_recall: 0.9943
Epoch 4/10
953/953 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.8724 - loss: 0.3299 - precision: 0.8758 - recall: 0.9945 - val_accuracy: 0.8716 - val_loss: 0.3236 - val_precision: 0.8744 - val_recall: 0.9958
Epoch 5/10
953/953 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.8758 - loss: 0.3184 - precision: 0.8791 - recall: 0.9948 - val_accuracy: 0.8716 - va

In [13]:
y_probs = model.predict(X_test).ravel()
y_pred = (y_probs > 0.5).astype(int)

# 8. Resultados
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step
Confusion Matrix:
 [[  88 1113]
 [  97 8224]]

Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.07      0.13      1201
           1       0.88      0.99      0.93      8321

    accuracy                           0.87      9522
   macro avg       0.68      0.53      0.53      9522
weighted avg       0.83      0.87      0.83      9522

F1 Score: 0.9314758183259713


## Tunner 1

In [17]:
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = {0: weights[0], 1: weights[1]}

print(class_weight)

{0: np.float64(3.9088669950738915), 1: np.float64(0.5733381502890174)}


In [ ]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = {0: weights[0], 1: weights[1]}


def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(hp.Int('units1', 32, 128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout1', 0.0, 0.5, step=0.1)))
    model.add(Dense(hp.Int('units2', 16, 64, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=[Precision()]
    )
    return model


from keras_tuner.engine import tuner as kt_engine

class MyTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['class_weight'] = class_weight
        return super().run_trial(trial, *args, **kwargs)

tuner = MyTuner(
    build_model,
    objective='val_precision',  
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory='tuning_dir',
    project_name='maximize_precision'
)


tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=1)


best_model = tuner.get_best_models(1)[0]


y_pred = (best_model.predict(X_val).ravel() > 0.5).astype(int)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

Trial 10 Complete [00h 00m 08s]
val_precision: 0.9435982704162598

Best val_precision So Far: 0.9435982704162598
Total elapsed time: 00h 01m 15s
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 279us/step
[[ 827  374]
 [2064 6257]]
              precision    recall  f1-score   support

           0       0.29      0.69      0.40      1201
           1       0.94      0.75      0.84      8321

    accuracy                           0.74      9522
   macro avg       0.61      0.72      0.62      9522
weighted avg       0.86      0.74      0.78      9522



/Users/martinagarciagonzalez/Library/Python/3.12/lib/python/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

{'units1': 128,
 'dropout1': 0.30000000000000004,
 'units2': 32,
 'lr': 0.00359800375100564}

In [50]:
model = build_model(best_hp)
model.fit(X_train, y_train, 
          epochs=32, 
          validation_data=(X_val, y_val), 
          class_weight=class_weight)

Epoch 1/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 536us/step - accuracy: 0.7202 - loss: 0.6148 - val_accuracy: 0.7546 - val_loss: 0.5362
Epoch 2/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - accuracy: 0.7590 - loss: 0.5800 - val_accuracy: 0.7432 - val_loss: 0.6037
Epoch 3/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 485us/step - accuracy: 0.7535 - loss: 0.5717 - val_accuracy: 0.7433 - val_loss: 0.6102
Epoch 4/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 460us/step - accuracy: 0.7599 - loss: 0.5671 - val_accuracy: 0.7469 - val_loss: 0.5585
Epoch 5/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 459us/step - accuracy: 0.7548 - loss: 0.5621 - val_accuracy: 0.7507 - val_loss: 0.5623
Epoch 6/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 575us/step - accuracy: 0.7577 - loss: 0.5617 - val_accuracy: 0.7444 - val_loss: 0.5644
Epoch 7/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 464us/step - accuracy: 0.7551 - loss: 0.5531 - val_accuracy: 0.7365 - val_loss: 0.6212
Epoch 8/32
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 460us/step - accuracy: 0.7480 -

In [51]:
# Predecir sobre el conjunto de validación
y_probs = model.predict(X_val).ravel()
y_pred = (y_probs > 0.5).astype(int)

# Métricas de evaluación
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

print(f"\nF1 Score: {f1_score(y_val, y_pred):.4f}")

298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step
Confusion Matrix:
[[ 840  361]
 [2537 5784]]

Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.70      0.37      1201
           1       0.94      0.70      0.80      8321

    accuracy                           0.70      9522
   macro avg       0.59      0.70      0.58      9522
weighted avg       0.85      0.70      0.75      9522


F1 Score: 0.7997


In [54]:
model = build_model(best_hp)
model.fit(X_train, y_train, 
          epochs=10, 
          validation_data=(X_val, y_val), 
          class_weight=class_weight)

Epoch 1/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 545us/step - accuracy: 0.7142 - loss: 0.6780 - val_accuracy: 0.7706 - val_loss: 0.5254
Epoch 2/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 481us/step - accuracy: 0.7588 - loss: 0.6008 - val_accuracy: 0.7538 - val_loss: 0.5775
Epoch 3/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 473us/step - accuracy: 0.7523 - loss: 0.5995 - val_accuracy: 0.7573 - val_loss: 0.5295
Epoch 4/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - accuracy: 0.7542 - loss: 0.5982 - val_accuracy: 0.7534 - val_loss: 0.6467
Epoch 5/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.7591 - loss: 0.5894 - val_accuracy: 0.7569 - val_loss: 0.5878
Epoch 6/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 493us/step - accuracy: 0.7529 - loss: 0.5860 - val_accuracy: 0.7507 - val_loss: 0.6119
Epoch 7/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 500us/step - accuracy: 0.7540 - loss: 0.5912 - val_accuracy: 0.7561 - val_loss: 0.5322
Epoch 8/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 1s 489us/step - accuracy: 0.7522 -

In [55]:
y_probs = model.predict(X_val).ravel()
y_pred = (y_probs > 0.5).astype(int)

# Métricas de evaluación
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

print(f"\nF1 Score: {f1_score(y_val, y_pred):.4f}")

298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step
Confusion Matrix:
[[ 811  390]
 [1981 6340]]

Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.68      0.41      1201
           1       0.94      0.76      0.84      8321

    accuracy                           0.75      9522
   macro avg       0.62      0.72      0.62      9522
weighted avg       0.86      0.75      0.79      9522


F1 Score: 0.8425


In [22]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Probabilidades predichas
y_probs = model.predict(X_val).ravel()

# Lista de umbrales a evaluar
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

for t in thresholds:
    print(f"\n🔎 UMBRAL: {t}")
    y_pred = (y_probs > t).astype(int)
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred))
    
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, digits=3))
    
    print(f"F1 Score: {f1_score(y_val, y_pred):.4f}")

298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 235us/step

🔎 UMBRAL: 0.3
Confusion Matrix:
[[ 598  603]
 [1393 6928]]

Classification Report:
              precision    recall  f1-score   support

           0      0.300     0.498     0.375      1201
           1      0.920     0.833     0.874      8321

    accuracy                          0.790      9522
   macro avg      0.610     0.665     0.624      9522
weighted avg      0.842     0.790     0.811      9522

F1 Score: 0.8741

🔎 UMBRAL: 0.4
Confusion Matrix:
[[ 734  467]
 [1776 6545]]

Classification Report:
              precision    recall  f1-score   support

           0      0.292     0.611     0.396      1201
           1      0.933     0.787     0.854      8321

    accuracy                          0.764      9522
   macro avg      0.613     0.699     0.625      9522
weighted avg      0.853     0.764     0.796      9522

F1 Score: 0.8537

🔎 UMBRAL: 0.5
Confusion Matrix:
[[ 840  361]
 [2481 5840]]

Classification Report:
              pre

## Fijar Nosotros ClassWeight

In [53]:
pesos_0 = [2.0, 3.0, 4.0, 5.0, 6.0]

# Entrenar y evaluar para cada peso
for peso_0 in pesos_0:
    print(f"\n🔎 Probando class_weight: {{0: {peso_0}, 1: 1.0}}")

    # Crear modelo con tus hiperparámetros óptimos
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=0.0036),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        class_weight={0: peso_0, 1: 1.0},
        verbose=0
    )

    # Evaluar
    y_pred_prob = model.predict(X_val).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    precision_0 = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0

    print(f"Precision clase 0: {precision_0:.3f}")
    print(f"Recall clase 0:    {recall_0:.3f}")
    print("Matriz de confusión:")
    print(cm)


🔎 Probando class_weight: {0: 2.0, 1: 1.0}
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 259us/step
Precision clase 0: 0.379
Recall clase 0:    0.183
Matriz de confusión:
[[ 220  981]
 [ 360 7961]]

🔎 Probando class_weight: {0: 3.0, 1: 1.0}
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step
Precision clase 0: 0.321
Recall clase 0:    0.474
Matriz de confusión:
[[ 569  632]
 [1205 7116]]

🔎 Probando class_weight: {0: 4.0, 1: 1.0}
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step
Precision clase 0: 0.295
Recall clase 0:    0.615
Matriz de confusión:
[[ 739  462]
 [1766 6555]]

🔎 Probando class_weight: {0: 5.0, 1: 1.0}
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 262us/step
Precision clase 0: 0.297
Recall clase 0:    0.673
Matriz de confusión:
[[ 808  393]
 [1914 6407]]

🔎 Probando class_weight: {0: 6.0, 1: 1.0}
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step
Precision clase 0: 0.296
Recall clase 0:    0.670
Matriz de confusión:
[[ 805  396]
 [1913 6408]]
